In [ ]:
import numpy as np
import os
import plotly.graph_objects as go

from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results
from chimera_fgo.util.plot import plot_trajectories

%load_ext autoreload
%autoreload 2

In [ ]:
kitti_seq = '0034'
MAX_BIAS = 1
start_idx = 1550 if kitti_seq == '0028' else 0

In [ ]:
# Load ground truth
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

Plot spoofed trajectories

In [ ]:
TRAJLEN = 1900
gps_spoofing_biases = np.zeros(TRAJLEN)  
attack_start_idx = 1000
gps_spoofing_biases[attack_start_idx:] = MAX_BIAS * np.linspace(0, 1, TRAJLEN-attack_start_idx)  # Ramping attack

spoof_pos_1 = gt_enu[:TRAJLEN].copy()
spoof_pos_1[:,0] += 50 * gps_spoofing_biases
spoof_pos_2 = gt_enu[:TRAJLEN].copy()
spoof_pos_2[:,0] += 100 * gps_spoofing_biases
spoof_pos_3 = gt_enu[:TRAJLEN].copy()
spoof_pos_3[:,0] += 200 * gps_spoofing_biases

gt_traj = go.Scatter(x=gt_enu[:TRAJLEN,0], y=gt_enu[:TRAJLEN,1], hovertext=np.arange(TRAJLEN), name='Ground-truth', line=dict(color='black'))
spoof_traj_1 = go.Scatter(x=spoof_pos_1[:,0], y=spoof_pos_1[:,1], hovertext=np.arange(TRAJLEN), name='0.5 m/s spoofed', line=dict(color='red', dash='dot'))
spoof_traj_2 = go.Scatter(x=spoof_pos_2[:,0], y=spoof_pos_2[:,1], hovertext=np.arange(TRAJLEN), name='1.0 m/s spoofed', line=dict(color='red', dash='dashdot'))
spoof_traj_3 = go.Scatter(x=spoof_pos_3[:,0], y=spoof_pos_3[:,1], hovertext=np.arange(TRAJLEN), name='2.0 m/s spoofed', line=dict(color='red', dash='dash'))

start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='black'), showlegend=False)
spoof_start = go.Scatter(x=[gt_enu[attack_start_idx,0]], y=[gt_enu[attack_start_idx,1]], 
            name='Spoofing start', mode='markers', marker=dict(size=10, color='red'), showlegend=True)
fig = go.Figure(data=[gt_traj, spoof_traj_1, spoof_traj_2, spoof_traj_3, start, spoof_start])
fig.update_layout(width=1000, height=1000, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
#fig.update_xaxes(autorange=True)
fig.update_xaxes(range=[-50, 950])
fig.show()

Plot Lidar odometry, blind FGO, and FGO

In [ ]:
# Lidar odometry
data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'icp')
lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

In [ ]:
# Load results
run_name = 'fgo_100m_5runs_2023-01-19-0507'
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)
fgo_results = np.load(os.path.join(results_path, results_files[0]))
fgo_positions = fgo_results['positions']

run_name = 'fgo_100m_5runs_blind_2023-01-19-0528'
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)
fgo_blind_results = np.load(os.path.join(results_path, results_files[0]))
fgo_blind_positions = fgo_blind_results['positions']

spoofed_positions = fgo_results['spoofed']

In [ ]:
# Plot
ATTACK_START = 1000
N = 1900
fgo_traj = go.Scatter(x=fgo_positions[:,0], y=fgo_positions[:,1], name='FGO', line=dict(color='blue'))
fgo_blind_traj = go.Scatter(x=fgo_blind_positions[:,0], y=fgo_blind_positions[:,1], name='FGO blind', line=dict(color='orange'))
gt_traj = go.Scatter(x=gt_enu[:N,0], y=gt_enu[:N,1], name='Ground-truth', line=dict(color='black'))
lidar_traj = go.Scatter(x=lidar_positions[:N,0], y=lidar_positions[:N,1], name='Lidar odometry', line=dict(color='green'))
start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='blue'), showlegend=False)
plot_data = [fgo_traj, fgo_blind_traj, lidar_traj, gt_traj, start]

spoof_traj = go.Scatter(x=spoofed_positions[:,0], y=spoofed_positions[:,1], 
    name='Spoofed', line=dict(color='red', dash='dash')) 
spoof_start = go.Scatter(x=[spoofed_positions[ATTACK_START,0]], y=[spoofed_positions[ATTACK_START,1]], 
    name='Spoofing start', mode='markers', marker=dict(size=10, color='red'), showlegend=False)
plot_data += [spoof_traj, spoof_start]


# detect = go.Scatter(x=[graph_positions[detect_idx,0]], y=[graph_positions[detect_idx,1]], 
#     name='Detection', mode='markers', hovertext=str(detect_idx), marker=dict(size=10, color='green'), showlegend=True)
# plot_data += [detect]

fig = go.Figure(data=plot_data)
fig.update_layout(width=1000, height=1000, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
fig.update_layout(legend=dict(x=0.02, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()

Plot test statistic

In [ ]:
kitti_seq = '0018'
run_name = 'fgo_0m_100runs_blind_2023-01-19-2054'
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)
fgo_blind_results = np.load(os.path.join(results_path, results_files[0]))

In [ ]:
#qs_plot = fgo_results['qs']
qs_plot = fgo_blind_results['qs']
T = fgo_results['threshold']
GPS_RATE = 10
detect_idx = np.argmax(qs_plot > T)

# Plot test statistic and threshold
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=qs_plot, name='Test statistic'))
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=T*np.ones(len(qs_plot)), name='Threshold', line=dict(color='black', dash='dash')))
# Add vertical line at start of spoofing attack
fig.add_trace(go.Scatter(x=[100, 100], y=[min(qs_plot), max(qs_plot)], mode='lines', name='Start of attack', line=dict(color='red', dash='dash')))
# fig.add_shape(type="line", x0=ATTACK_START/GPS_RATE, y0=-20, x1=ATTACK_START/GPS_RATE, y1=200, name='Start of attack', 
#               line=dict(color="red", width=2, dash="dash"), showlegend=True)
fig.add_trace(go.Scatter(x=[detect_idx], y=[qs_plot[detect_idx]], name='Detection', mode='markers', marker=dict(size=10, color='red'), showlegend=True))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Test statistic')
fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))
fig.show()

In [ ]:
n_runs = len(results_files)

fig = go.Figure()

per_trial_FA = 0
per_run_FA = 0

for i in range(n_runs):
    results = np.load(os.path.join(results_path, results_files[i]))
    qs = results['qs']
    T = results['threshold']
    detect_idx = np.argmax(qs > T)
    showlegend = True if i == 0 else False
    fig.add_trace(go.Scatter(x=np.arange(len(qs)), y=qs, name='Test statistic', line=dict(color='blue'), showlegend=showlegend))
    #fig.add_trace(go.Scatter(x=[detect_idx], y=[qs[detect_idx]], name='Detection', mode='markers', marker=dict(size=10, color='red'), showlegend=showlegend))
    per_trial_FA += sum(qs > T)
    per_run_FA += 1 if sum(qs > T) > 0 else 0

fig.add_trace(go.Scatter(x=np.arange(len(qs)), y=T*np.ones(len(qs)), name='Threshold', line=dict(color='black', dash='dash')))
#fig.add_trace(go.Scatter(x=[100, 100], y=[50, 400], mode='lines', name='Start of attack', line=dict(color='red', dash='dash')))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Test statistic')
fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))
fig.update_yaxes(range=[50, 200])
fig.show()

In [ ]:
per_trial_FA

In [ ]:
per_run_FA

Plot error over time

In [ ]:
kitti_seq = '0027'
fgo_run = 'fgo_-200m_20runs_100w_2023-01-20-2106'
fgo_blind_run = 'fgo_-200m_20runs_blind_2023-01-20-2223'
N = 1900

In [ ]:
import plotly.express as px
colors = px.colors.qualitative.D3

In [ ]:
fgo_results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, fgo_run)
fgo_results_files = os.listdir(fgo_results_path)

fgo_blind_results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, fgo_blind_run)
fgo_blind_results_files = os.listdir(fgo_blind_results_path)

start_idx = 1550 if kitti_seq == '0028' else 0
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)
data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'icp')
lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

n_runs = len(fgo_blind_results_files)
fgo_blind_err_avg = np.zeros(1900)
fgo_err_avg = np.zeros(1900)
time = np.arange(1900) / 10

fig = go.Figure()
fig.add_trace(go.Scatter(x=[100, 100], y=[0, 300], mode='lines', name='Start of attack', line=dict(color='red', dash='dash'), showlegend=False))

for i in range(n_runs):
    fgo_results = np.load(os.path.join(fgo_results_path, fgo_results_files[i]))
    fgo_blind_results = np.load(os.path.join(fgo_blind_results_path, fgo_blind_results_files[i]))
    fgo_positions = fgo_results['positions']
    fgo_blind_positions = fgo_blind_results['positions']
    
    fgo_blind_err = np.linalg.norm(fgo_blind_positions - gt_enu[:N], axis=1)
    fgo_err = np.linalg.norm(fgo_positions - gt_enu[:N], axis=1)
    # fgo_blind_err_avg += fgo_blind_err
    # fgo_err_avg += fgo_err
    showlegend = True if i == 0 else False
    fig.add_trace(go.Scatter(x=time, y=fgo_blind_err, name='Naive FGO', line=dict(color='orange'), showlegend=showlegend))
    fig.add_trace(go.Scatter(x=time, y=fgo_err, name='SR FGO', line=dict(color='blue'), showlegend=showlegend))


lidar_err = np.linalg.norm(lidar_positions[:N] - gt_enu[:N], axis=1)
fgo_blind_err_avg /= n_runs
fgo_err_avg /= n_runs

fig.add_trace(go.Scatter(x=time, y=lidar_err, name='Odometry only', line=dict(color='green')))
# fig.add_trace(go.Scatter(x=np.arange(len(fgo_blind_err_avg)), y=fgo_blind_err_avg, name='FGO blind'))
# fig.add_trace(go.Scatter(x=np.arange(len(fgo_err_avg)), y=fgo_err_avg, name='FGO'))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='L2 norm error [m]')
fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))

Normalized error scatter plot

In [ ]:
N = 1900
lidar_mean_errs = []
lidar_max_errs = []

for kitti_seq in ['0018', '0027', '0028', '0034']:

    start_idx = 1550 if kitti_seq == '0028' else 0
    gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
    gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)
    data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'icp')
    lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

    # Compute RMSE and max error for lidar for each sequence
    lidar_err = np.linalg.norm(lidar_positions[:N] - gt_enu[:N], axis=1)
    print("Mean error: ", np.mean(lidar_err))
    print("Max error: ", np.max(lidar_err))
    lidar_mean_errs.append(np.mean(lidar_err))
    lidar_max_errs.append(np.max(lidar_err))

In [ ]:
runs = [['fgo_0m_10runs_2023-01-18-1754', 'fgo_50m_5runs_2023-01-19-0042', 'fgo_100m_5runs_2023-01-19-0121', 'fgo_200m_20runs_2023-01-19-2015'], # 0018
        ['fgo_0m_10runs_2023-01-18-1843', 'fgo_50m_5runs_2023-01-19-0237', 'fgo_100m_5runs_2023-01-19-0313', 'fgo_200m_20runs_2023-01-19-2308'], # 0027
        ['fgo_0m_10runs_2023-01-18-1920', 'fgo_50m_5runs_2023-01-19-0428', 'fgo_100m_5runs_2023-01-19-0507', 'fgo_200m_20runs_2023-01-20-0138'], # 0028
        ['fgo_0m_10runs_2023-01-18-2000', 'fgo_50m_5runs_2023-01-19-0627', 'fgo_100m_5runs_2023-01-19-0707', 'fgo_200m_20runs_100w_2023-01-20-0846']] # 0034
mean_errors = np.zeros((4, 4))
max_errors = np.zeros((4, 4))
mean_errors_std = np.zeros((4, 4))
max_errors_std = np.zeros((4, 4))

runs_blind = [['fgo_0m_100runs_blind_2023-01-19-2054', 'fgo_50m_5runs_blind_2023-01-19-0102', 'fgo_100m_20runs_blind_2023-01-20-1012', 'fgo_200m_20runs_blind_2023-01-19-2142'], # 0018
        ['fgo_0m_100runs_blind_2023-01-18-2330', 'fgo_50m_5runs_blind_2023-01-19-0255', 'fgo_100m_20runs_blind_2023-01-20-1251', 'fgo_200m_20runs_blind_2023-01-20-0025'], # 0027
        ['fgo_0m_100runs_blind_2023-01-18-2330', 'fgo_50m_5runs_blind_2023-01-19-0448', 'fgo_100m_20runs_blind_2023-01-20-1526', 'fgo_200m_20runs_blind_2023-01-20-0300'], # 0028
        ['fgo_0m_100runs_blind_2023-01-18-2330', 'fgo_50m_5runs_blind_2023-01-19-0647', 'fgo_100m_20runs_blind_2023-01-20-1810', 'fgo_200m_20runs_blind_2023-01-20-1013']] # 0034
mean_errors_blind = np.zeros((4, 4))
max_errors_blind = np.zeros((4, 4))
mean_errors_blind_std = np.zeros((4, 4))
max_errors_blind_std = np.zeros((4, 4))

In [ ]:
traces = ['0018', '0027', '0028', '0034']
for i in range(4):
    kitti_seq = traces[i]
    start_idx = 1550 if kitti_seq == '0028' else 0
    gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
    gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)
    for j in range(4):
        # SR FGO
        run = runs[i][j]
        fgo_results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run)
        fgo_results_files = os.listdir(fgo_results_path)

        mean_errs = []
        max_errs = []
        for k in range(5):
            fgo_results = np.load(os.path.join(fgo_results_path, fgo_results_files[k]))
            fgo_positions = fgo_results['positions']
            err = np.linalg.norm(fgo_positions - gt_enu[:N], axis=1)
            # print("Mean error: ", np.mean(err))
            # print("Max error: ", np.max(err))
            mean_errs.append(np.mean(err))
            max_errs.append(np.max(err))
        mean_errs_mean = np.mean(mean_errs)
        mean_errs_std = np.std(mean_errs)
        max_errs_mean = np.mean(max_errs)
        max_errs_std = np.std(max_errs)
        mean_errors[i,j] = mean_errs_mean #/ lidar_mean_errs[i]
        max_errors[i,j] = max_errs_mean #/ lidar_max_errs[i]
        mean_errors_std[i,j] = mean_errs_std
        max_errors_std[i,j] = max_errs_std

        # Naive FGO
        run_blind = runs_blind[i][j]
        fgo_results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_blind)
        fgo_results_files = os.listdir(fgo_results_path)

        mean_errs = []
        max_errs = []
        for k in range(5):
            fgo_results = np.load(os.path.join(fgo_results_path, fgo_results_files[k]))
            fgo_positions = fgo_results['positions']
            err = np.linalg.norm(fgo_positions - gt_enu[:N], axis=1)
            # print("Mean error: ", np.mean(err))
            # print("Max error: ", np.max(err))
            mean_errs.append(np.mean(err))
            max_errs.append(np.max(err))
        mean_errs_mean = np.mean(mean_errs)
        mean_errs_std = np.std(mean_errs)
        max_errs_mean = np.mean(max_errs)
        max_errs_std = np.std(max_errs)
        mean_errors_blind[i,j] = mean_errs_mean #/ lidar_mean_errs[i]
        max_errors_blind[i,j] = max_errs_mean #/ lidar_max_errs[i]
        mean_errors_blind_std[i,j] = mean_errs_std
        max_errors_blind_std[i,j] = max_errs_std

In [ ]:
# mean error plot
seq_i = 2
fig = go.Figure()
fig.add_trace(go.Scatter(x=['0 m/s', '0.5 m.s', '1.0 m/s', '2.0 m/s'], y=4*[lidar_mean_errs[seq_i]], name='Lidar', mode='lines', line=dict(color='green', width=2, dash='dash')))
fig.add_trace(go.Scatter(x=['0 m/s', '0.5 m.s', '1.0 m/s', '2.0 m/s'], y=mean_errors[seq_i], error_y=dict(type='data', array=mean_errors_std[seq_i], visible=True), 
    name='SR FGO', mode='markers', marker=dict(color='blue', size=10, symbol='diamond')))
fig.add_trace(go.Scatter(x=['0 m/s', '0.5 m.s', '1.0 m/s', '2.0 m/s'], y=mean_errors_blind[seq_i], error_y=dict(type='data', array=mean_errors_blind_std[seq_i], visible=True),
    name='Naive FGO', mode='markers', marker=dict(color='red', size=10, symbol='square')))
fig.update_layout(width=900, height=500, xaxis_title='Spoofing rate', yaxis_title='Mean error (m)')
fig.update_layout(legend=dict(x=1.02, y=0.98), font=dict(size=18))

In [ ]:
# max error plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=['0 m/s', '0.5 m.s', '1.0 m/s', '2.0 m/s'], y=4*[lidar_max_errs[seq_i]], name='Lidar', mode='lines', line=dict(color='green', width=2, dash='dash')))
fig.add_trace(go.Scatter(x=['0 m/s', '0.5 m.s', '1.0 m/s', '2.0 m/s'], y=max_errors[seq_i], error_y=dict(type='data', array=max_errors_std[seq_i], visible=True), 
    name='SR FGO', mode='markers', marker=dict(color='blue', size=10, symbol='diamond')))
fig.add_trace(go.Scatter(x=['0 m/s', '0.5 m.s', '1.0 m/s', '2.0 m/s'], y=max_errors_blind[seq_i], error_y=dict(type='data', array=max_errors_blind_std[seq_i], visible=True),
    name='Naive FGO', mode='markers', marker=dict(color='red', size=10, symbol='square')))
fig.update_layout(width=900, height=500, xaxis_title='Spoofing rate', yaxis_title='Max error (m)')
fig.update_layout(legend=dict(x=1.02, y=0.98), font=dict(size=18))

Window size comparison

In [ ]:
kitti_seq = '0027'
start_idx = 0
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)
runs = ['fgo_100m_5runs_20w_2023-01-19-2312', 
        'fgo_100m_5runs_50w_2023-01-19-2314',
        'fgo_100m_5runs_100w_2023-01-19-2321',
        'fgo_100m_5runs_200w_2023-01-19-2340']

mean_errs = []
max_errs = []
avg_iter_times = []

for run in runs:
    fgo_results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run)
    fgo_results_files = os.listdir(fgo_results_path)
    run_mean_errs = []
    run_max_errs = []
    run_avg_iter_times = []
    for k in range(5):
        fgo_results = np.load(os.path.join(fgo_results_path, fgo_results_files[k]))
        fgo_positions = fgo_results['positions']
        N = len(fgo_positions)
        err = np.linalg.norm(fgo_positions - gt_enu[:N], axis=1)
        run_mean_errs.append(np.mean(err))
        run_max_errs.append(np.max(err))
        run_avg_iter_times.append(fgo_results['avg_iter_time'])
    mean_errs.append(np.mean(run_mean_errs))
    max_errs.append(np.mean(run_max_errs))
    avg_iter_times.append(np.mean(run_avg_iter_times))

In [ ]:
print("Window sizes: ", [20, 50, 100, 200])
print("Mean error: ", mean_errs)
print("Max error: ", max_errs)
print("Avg iter time: ", avg_iter_times)

Monte carlo trajectory plots

In [ ]:
kitti_seq = '0028'
run_name = 'fgo_100m_5runs_2023-01-19-0507'

In [ ]:
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)

start_idx = 1550 if kitti_seq == '0028' else 0
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

N_SHIFT = 10

for fname in results_files:
    results = np.load(os.path.join(results_path, fname))
    graph_positions = results['positions']
    gt_enu = gt_enu[:graph_positions.shape[0]]
    if MAX_BIAS != 0:
        spoofed_positions = results['spoofed']
    else: 
        spoofed_positions = None

    qs = results['qs']
    threshold = results['threshold']
    if any(qs > threshold):
        detect_idx = N_SHIFT * np.argmax(qs > threshold)
    else:
        detect_idx = None

    plot_trajectories(gt_enu, graph_positions, spoofed_positions, detect_idx)

Monte carlo errors

In [ ]:
from plotly.subplots import make_subplots

def plot_errors(fig, gt_enu, graph_positions):
    traj_len = len(gt_enu)
    
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,0] - gt_enu[:,0], name='x error'), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,1] - gt_enu[:,1], name='y error'), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,2] - gt_enu[:,2], name='z error'), row=3, col=1)

In [ ]:
# Error envelope
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)

for fname in results_files:
    results = np.load(os.path.join(results_path, fname))
    graph_positions = results['positions']
    gt_enu = gt_enu[:graph_positions.shape[0]]
    plot_errors(fig, gt_enu, graph_positions)

fig.update_layout(width=1200, height=700)
fig.update_layout(font=dict(size=15))
fig.show()

Error statistics

In [ ]:
np.set_printoptions(suppress=True, precision=3)

In [ ]:
# Nominal

false_alarms = 0
total_trials = 0
trajectory_false_alarms = 0

ss_start = 0  # Start of steady state
xyz_errors = []

for fname in results_files:
    results = np.load(os.path.join(results_path, fname))

    graph_positions = results['positions']
    gt_enu = gt_enu[:graph_positions.shape[0]]

    xyz_errors.append(gt_enu[ss_start:] - graph_positions[ss_start:])

    # False alarm
    qs = results['qs']
    threshold = results['threshold']
    false_alarms += np.sum(qs > threshold)
    total_trials += len(qs)
    if any(qs > threshold):
        trajectory_false_alarms += 1


print(f'Per trial false alarm rate: {false_alarms} of {total_trials}')
print(f'Per trajectory false alarm rate: {trajectory_false_alarms} of {len(results_files)}')

In [ ]:
xyz_errors = np.vstack(xyz_errors)

In [ ]:
print("mean: ", np.mean(xyz_errors, axis=0))
print("std: ", np.std(xyz_errors, axis=0))